## **Toxic Comment Classification:**

**Pipeline:**

* import libraries
* Pre-processing:
    * lower case 
    * decide if to replace \n by space or period
    * remove words that have numbers inside them 
    * remove any other non a-z charachter
* RE:
    * Remove non letter characters
    * Convert to lowercase
* We may want to use a trained model instead of training on this limied dataset.  




1. Which word embedding methodology to use? BOW (Bag Of Word),Word2Vec,Doc2Vec,FastText,TFIDF
2. Strategy\plan how to clean the vocabulary
3. Direct feaures from the text
4. Classification of 6 categories
5. Apply word vectors on comments
6. EDA: correlation between the columns
7. Kaggle collaborate on notebook

Direct features:
1. contains question Mark
2. contains exclamation mark (especially multiple - and count them)
3. contains words with all capital
4. begins with hi (or similar)
5. begins with 'I'
6. begins with 'You'
7. begins with a capital or small letter
8. contains parenthesis
9. conains curses from a glosary: fuck, shit, ass, looser, dick, bitch, suck, Nazi,stupid, bullshit, piss, cock, jew, die, cunt, rape, gay, cut, looser, nigger
10. contains negative words: bad, terrible, kill, block, stop, wrong, destroy, absurd, don't, mom, mother, fat
12. conains A$$HOLE, NIGGA, FVCK, cra*p, fxxk
13. abbreviations: OMG, LMAO, WTF, FFS

Remove:
1. Multiple spaces
2. Usernames: JacobBarnett52698
3. email addresses
4. '=', '-', 
    
Terms:
* tockens
* corpus
* NLP




In [ ]:
# General tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
import re
import sys
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import tensorflow as tf
from sklearn.manifold import TSNE # t-distributed Stochastic Neighbor Embedding; to visualize high-dimensional data. 
from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features.
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
from wordcloud import WordCloud, STOPWORDS

### **Get the data:**

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

traincomments = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
traincomments['comment_characters']=traincomments['comment_text'].str.len()
traincomments['comment_words']=traincomments['comment_text'].str.split().str.len()
traincomments['number_categories']= traincomments.iloc[:, 2:8].sum(axis=1)
traincomments['exclamation_mark_count']=traincomments['comment_text'].str.count('!')
traincomments['question_mark_count']=traincomments['comment_text'].str.count('\?')

def find_exclamation_mark_cosecutive y=re.search(['e']{2,}
                                                 
traincomments['exclamation_mark_cosecutive']=traincomments['comment_text'].apply(find_exclamation_mark_cosecutive(x))
                                                                                 
traincomments['all_upper']=traincomments['comment_text'].str.split().apply(lambda x: np.any([True if w.isupper() else False for w in x]))
traincomments['begins_greeting'] = traincomments['comment_text'].apply(lambda x: True if re.match('^hey|hi|hello', x, re.IGNORECASE) else False)
traincomments['begins_I'] = traincomments['comment_text'].apply(lambda x: True if re.match('^I ', x, re.IGNORECASE) else False)
traincomments['begins_You'] = traincomments['comment_text'].apply(lambda x: True if re.match('^You', x, re.IGNORECASE) else False)
traincomments['begins_capital'] = traincomments['comment_text'].apply(lambda x: True if re.match('^[A-Z].*', x) else False)
traincomments['contains_parenthesis'] = traincomments['comment_text'].apply(lambda x: True if re.search('\(|\)', x) else False)
traincomments['contains_curse'] = traincomments['comment_text'].apply(lambda x: True if re.search('fuck|shit |ass| looser | dick|bitch|suck|Nazi|stupid|bullshit| piss | cock|jew| die | cunt| rape | gay | nigger | pussy|wank|faggot|dumb', x,re.IGNORECASE) else False)
traincomments['contains_curse_replacement'] = traincomments['comment_text'].apply(lambda x: True if re.search('A$$HOLE|NIGGA|FVCK|cra*p|fxxk|b*tch', x,re.IGNORECASE) else False)
traincomments['contains_negative_terms'] = traincomments['comment_text'].apply(lambda x: True if re.search(' bad | terrible | kill | block | stop | wrong | destroy | absurd | don\'t | mom | mother | fat | cut | burn | ugly | kick | liar ', x,re.IGNORECASE) else False)
traincomments['contains_abbreviations'] = traincomments['comment_text'].apply(lambda x: True if re.search(' OMG | LMAO |WTF|FFS', x,re.IGNORECASE) else False)

traincomments.head(7)

### **Data Exploration:**

In [ ]:
traincomments.shape
traincomments.nunique()
traincomments.info()
traincomments['toxic'].value_counts().head(10) #Count entries per value:

### **Preliminary EDA**

In [ ]:
sums=traincomments[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].sum(axis=0)
sums_df=pd.DataFrame(data=sums, columns=['number'])


fig = plt.figure()
fig.set_size_inches(12,10)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax1.set_title("Comment # characters distribution", x=0.4, y=0.95, ha='center', fontsize='xx-large');
ax2.set_title("Category breakdown", x=0.4, y=0.95, ha='center', fontsize='xx-large');
ax3.set_title("Comment # words distribution", x=0.4, y=0.95, ha='center', fontsize='xx-large');
ax4.set_title("Number of categories per comment", x=0.4, y=0.95, ha='center', fontsize='xx-large');

sns.distplot(traincomments.comment_characters, bins=100, kde=True, color='blue', ax=ax1);
ax1.set_xlim(-200,1000)
sums_df.sort_values(by='number',ascending=False).plot.bar(ax=ax2);
sns.distplot(traincomments.comment_words, bins=100, kde=True, color='blue', ax=ax3);
ax3.set_xlim(-25,300)
traincomments[traincomments['number_categories']>0]['number_categories'].value_counts().sort_index(ascending=True).plot.bar(ax=ax4);


plt.show();


In [ ]:
corr_df = traincomments[traincomments['number_categories']>0]
corr=corr_df.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


plt.figure(figsize=(10,8))
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True, mask=mask, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});


In [ ]:
traincomments_toxic=traincomments[traincomments['toxic']==1]
traincomments_severe_toxic=traincomments[traincomments['severe_toxic']==1]
traincomments_obscene=traincomments[traincomments['obscene']==1]
traincomments_threat=traincomments[traincomments['threat']==1]
traincomments_insult=traincomments[traincomments['insult']==1]
traincomments_identity_hate=traincomments[traincomments['identity_hate']==1]

text = traincomments_identity_hate['comment_text'].values
wordcloud = WordCloud(width = 3000,height = 2000,background_color = 'black',stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(figsize = (8, 8), facecolor = 'k',edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
text = traincomments['comment_text'].values

wordcloud = WordCloud(width = 3000,height = 2000,background_color = 'black',stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(figsize = (8, 8), facecolor = 'k',edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

### **Word2Vector - Gensim**

In [ ]:
words_with_numbers_pattern = re.compile(r'\w*[\d]+\w*') # letter\s (0 or more), digit (1 or more), letter\s (0 or more)
special_chars_pattern = re.compile(r'[,."!?:;=&*\\/()\'$^#]+')
def clean(s):
    s = re.sub(words_with_numbers_pattern, '', s).strip()  # get rid of numbers or words with numbers 
    s = re.sub(special_chars_pattern, '', s)  # remove special chars without removing words
    # ''.join([char.strip(',."!?:;=&*\\/()\'$^#') for char in s])  # remove these characters - this is so slow-- change to re
    return s

In [ ]:
comment_list = traincomments[:159571]['comment_text'].values # gives a list of all comments
comment_list=[c.lower() for c in comment_list] # need to do with "map" before
sentences_list = [re.split('[.\n!?]',line) for line in comment_list] # gives a list of list (each list is a list of sentences corresponding to 1 comment)
sentences = [s for sentence in sentences_list for s in sentence] # gives a list of all sentences; we loose the comment context
# print(len(sentences))  # after this split we have ~1.4M sentences
sentences = [clean(s) for s in sentences if len(s) > 1]
# print(len(sentences))  # now we have ~954K
sentences = [s.split() for s in sentences] # list of list; words within sentences
sentences = [words_list for words_list in sentences if len(words_list)>1] # remove lists with 1 word or less
# print(len(sentences))  # now we have ~771K

# maybe we want to change \n to space before
# we can use Pandas functions (map) to do the splits
# remove empty words 
# remove words with no letters
# clean special charachters

In [ ]:
model = Word2Vec(sentences, size=100, window=5, min_count=3, workers=4) # train the model


In [ ]:
sims=model.wv.most_similar('kill')
sims_df=pd.DataFrame(data=sims, columns=['word','score'])
sims_df.sort_values(by='score',ascending=False).plot.bar('word','score',ylim=(0.62,0.82));


In [ ]:
len(model.wv.vocab)

In [ ]:
ordered_vocab = [(term, voc.index, voc.count) for term, voc in model.wv.vocab.items()] # build tokens and counts list
ordered_vocab = sorted(ordered_vocab, key=lambda k: -k[2]) # sort by tokens counts desc
ordered_terms, term_indices, term_counts = zip(*ordered_vocab) # unzip the tokens, indices and counts into separate lists
word_vectors = pd.DataFrame(model.wv.syn0norm[term_indices, :], index=ordered_terms) # create a DataFrame
print(word_vectors[:15].index.tolist())

### **TSNE**

In [ ]:
how_many_to_use = 700
X = word_vectors[:how_many_to_use].to_numpy() # will use top most frequent
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)
df = pd.DataFrame(X_tsne, index=word_vectors[:how_many_to_use].index.tolist(), columns=['x', 'y'])
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(df['x'], df['y'])
for word, pos in df.iterrows():
    ax.annotate(word, pos)

In [ ]:
# code to add vector per comment to training df
# need to clean comment_list before this

wvc=word_vectors.columns.tolist()
cols=traincomments.columns.tolist()
cols.extend(wvc)

traincomments=traincomments.reindex(columns=cols)
traincomments.head()
# print(traincomments.columns.tolist())

for i, comment in enumerate(comment_list[0:100]):
    org_words = clean(comment).split()
    words = [w for w in org_words if w in word_vectors.index]  #remove words not in model
    comment_vector=word_vectors.loc[words].mean(axis=0)   # average the word vectors for the comment (cum then normalize for word nmber)
    traincomments.loc[i, wvc]=comment_vector  # add the features to the df

traincomments.head(23)



In [ ]:
def get_related_terms(token, topn=10):
    """
    look up the top most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in model.most_similar(positive=[token], topn=topn):
        print (word, round(similarity, 3))

### **Word2Vector - TensorFlow**

In [ ]:
corpus_raw = 'He is the king . The king is royal . She is the royal  queen '
corpus_raw = corpus_raw.lower() # convert to lower case

### **TFIDF**

In [ ]:
v = TfidfVectorizer()

X_train = v.fit_transform(train['comment_text'])
# X_test = v.transform(test['comment_text'])

for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    y = train[label]
    model = LogisticRegression()
    model.fit(X_train, y)
    test[label] = model.predict_proba(X_test)[:, 1]
    
test.drop('comment_text', axis=1, inplace=True)
test.to_csv('simplest.csv', index=False)

### **Bag of Words**

### **TFIDF**